In [1]:
import gym
import numpy as np
import torch
import torch.nn as nn
import random


torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make('CartPole-v0').unwrapped


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [2]:
input_size = 4
num_classes = 2
hidden_size = 20
learning_rate = 0.001
gamma = 0.9

hidden_size = 100


class NN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()        
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.out = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.out(out)
        return out
    
class DQNAgent:
    def __init__(self, capacity = 20000, input_size = 4, \
                 hidden_size = 10, num_classes = 2, learning_rate = 0.001, \
                epsilon = 0.9, epsilon_decay_rate = 0.9999, batch_size = 32, \
                gamma = 0.95):
        self.replay = deque(maxlen=capacity)
        self.epsilon = epsilon
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon_decay_rate = epsilon_decay_rate
        
        # Model initialization
        self.q_network = NN(input_size, hidden_size, num_classes)
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.q_network.parameters(), lr = learning_rate)
    
    def act(self, state):
        if random.random()<self.epsilon:
            return random.choice([0,1])
        state = torch.tensor(state, dtype = torch.float32)
        return np.argmax(self.q_network(state).detach().numpy())
    
    def get_q(self, state):
        state = torch.tensor(state, dtype = torch.float32)
        return self.q_network(state).detach().numpy()
    
    def push(self, state, action, reward, next_state, done):
        self.replay.append((state, action, reward, next_state, done))
    
    def train(self, freeze = True):
        if self.batch_size >= len(self.replay):
            return -1
        minibatch = random.sample(self.replay, self.batch_size)  
        minibatch = np.array(minibatch).T
        
        
        
        targets = minibatch[2]
        done = np.logical_not(minibatch[4])

        next_states = np.stack(minibatch[3])
        q_updates = np.max(self.get_q(next_states), axis = 1)

        targets [np.nonzero(done)]+= q_updates[np.nonzero(done)]
        
        states = np.stack(minibatch[0])
        states = torch.tensor(states, dtype=torch.float32, requires_grad = True)
        outputs = self.q_network(states)
        
        final_targets = copy.deepcopy(outputs.detach().numpy())
        actions = list(minibatch[1])

        
        final_targets[range(self.batch_size), actions] = targets
        
        


        loss = self.criterion(outputs, torch.tensor(final_targets, dtype = torch.float32))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.epsilon*=self.epsilon_decay_rate

In [3]:
# Architecture 1
import copy
from collections import deque

num_episodes = 10000
batch_size = 64

agent = DQNAgent()


for i in range(num_episodes):
    state = env.reset()
    done = False
    t = 0
    while not done:
        t+=1
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10 # reward shaping
        agent.push(state, action, reward, next_state, done)
        state = next_state
        debug = agent.train()
        if done:
            print("episode: {}/{}, score: {}, eps: {}" .format(i+1, num_episodes, t, agent.epsilon))
        

episode: 1/10000, score: 14, eps: 0.9
episode: 2/10000, score: 18, eps: 0.9
episode: 3/10000, score: 43, eps: 0.8961381159041989
episode: 4/10000, score: 23, eps: 0.8940792638807852
episode: 5/10000, score: 11, eps: 0.8930962682866179
episode: 6/10000, score: 15, eps: 0.8917575612290328
episode: 7/10000, score: 15, eps: 0.8904208608270006
episode: 8/10000, score: 28, eps: 0.8879310452923425
episode: 9/10000, score: 18, eps: 0.8863341272210353
episode: 10/10000, score: 13, eps: 0.8851825839428392
episode: 11/10000, score: 27, eps: 0.882795695369457
episode: 12/10000, score: 20, eps: 0.8810317802845797
episode: 13/10000, score: 27, eps: 0.878656084323888
episode: 14/10000, score: 22, eps: 0.8767250692814424
episode: 15/10000, score: 23, eps: 0.874710818184617
episode: 16/10000, score: 14, eps: 0.873487018707696
episode: 17/10000, score: 12, eps: 0.8724394105945552
episode: 18/10000, score: 13, eps: 0.8713059196140674
episode: 19/10000, score: 21, eps: 0.8694780057669936
episode: 20/10000

episode: 155/10000, score: 78, eps: 0.5281963642454592
episode: 156/10000, score: 73, eps: 0.5243543789926307
episode: 157/10000, score: 43, eps: 0.5221043836184135
episode: 158/10000, score: 72, eps: 0.5183585459597416
episode: 159/10000, score: 97, eps: 0.5133545265903174
episode: 160/10000, score: 30, eps: 0.511816694019924
episode: 161/10000, score: 154, eps: 0.5039947097011402
episode: 162/10000, score: 102, eps: 0.4988798381075517
episode: 163/10000, score: 161, eps: 0.4909117892228175
episode: 164/10000, score: 203, eps: 0.48104626053817845
episode: 165/10000, score: 122, eps: 0.4752128605815319
episode: 166/10000, score: 54, eps: 0.47265349965826053
episode: 167/10000, score: 60, eps: 0.46982592847607496
episode: 168/10000, score: 40, eps: 0.4679502847668233
episode: 169/10000, score: 139, eps: 0.46149045265827654
episode: 170/10000, score: 135, eps: 0.4553018889125556
episode: 171/10000, score: 181, eps: 0.44713465282351755
episode: 172/10000, score: 93, eps: 0.442995371080151

episode: 303/10000, score: 10, eps: 0.012000866901052199
episode: 304/10000, score: 8, eps: 0.011991269567102126
episode: 305/10000, score: 9, eps: 0.011980481740341664
episode: 306/10000, score: 10, eps: 0.011968506648380702
episode: 307/10000, score: 9, eps: 0.011957739300054351
episode: 308/10000, score: 10, eps: 0.011945786940302305
episode: 309/10000, score: 9, eps: 0.011935040031536037
episode: 310/10000, score: 10, eps: 0.011923110360840563
episode: 311/10000, score: 10, eps: 0.011911192614448866
episode: 312/10000, score: 10, eps: 0.011899286780442
episode: 313/10000, score: 10, eps: 0.011887392846912944
episode: 314/10000, score: 10, eps: 0.011875510801966575
episode: 315/10000, score: 11, eps: 0.011862454269656285
episode: 316/10000, score: 9, eps: 0.011851782330300837
episode: 317/10000, score: 10, eps: 0.011839935879850619
episode: 318/10000, score: 11, eps: 0.011826918460394317
episode: 319/10000, score: 10, eps: 0.011815096862628249
episode: 320/10000, score: 10, eps: 0.0

episode: 447/10000, score: 146, eps: 0.00031917923679502115
episode: 448/10000, score: 48, eps: 0.0003176507712858773
episode: 449/10000, score: 41, eps: 0.0003163510044769871
episode: 450/10000, score: 93, eps: 0.0003134224326719203
episode: 451/10000, score: 113, eps: 0.000309900519372158
episode: 452/10000, score: 106, eps: 0.0003066327601983436
episode: 453/10000, score: 35, eps: 0.00030556136799726594
episode: 454/10000, score: 16, eps: 0.0003050728363110532
episode: 455/10000, score: 12, eps: 0.00030470695018845105
episode: 456/10000, score: 11, eps: 0.0003043719400817998
episode: 457/10000, score: 11, eps: 0.00030403729830206556
episode: 458/10000, score: 13, eps: 0.0003036422868764327
episode: 459/10000, score: 12, eps: 0.00030327811646930407
episode: 460/10000, score: 13, eps: 0.00030288409138810895
episode: 461/10000, score: 14, eps: 0.0003024603291744693
episode: 462/10000, score: 10, eps: 0.0003021580049161542
episode: 463/10000, score: 13, eps: 0.0003017654351066116
episod

episode: 588/10000, score: 37, eps: 2.9012678173023594e-05
episode: 589/10000, score: 44, eps: 2.8885296465147447e-05
episode: 590/10000, score: 78, eps: 2.8660856384808543e-05
episode: 591/10000, score: 72, eps: 2.8455229083938266e-05
episode: 592/10000, score: 4416, eps: 1.8296501400970204e-05
episode: 593/10000, score: 132, eps: 1.8056562665072122e-05
episode: 594/10000, score: 17, eps: 1.802589105319256e-05
episode: 595/10000, score: 15, eps: 1.7998871135599057e-05
episode: 596/10000, score: 13, eps: 1.7975486637095875e-05
episode: 597/10000, score: 15, eps: 1.794854227322481e-05
episode: 598/10000, score: 15, eps: 1.792163829762023e-05
episode: 599/10000, score: 13, eps: 1.78983541415869e-05
episode: 600/10000, score: 12, eps: 1.7876887925593982e-05
episode: 601/10000, score: 10, eps: 1.7859019080123115e-05
episode: 602/10000, score: 11, eps: 1.7839383978649325e-05
episode: 603/10000, score: 12, eps: 1.7817988487944586e-05
episode: 604/10000, score: 11, eps: 1.7798398497562136e-05

...

episode: 9190/10000, score: 10, eps: 3.4324192996858983e-28
episode: 9191/10000, score: 9, eps: 3.4293313576988495e-28
episode: 9192/10000, score: 10, eps: 3.4259035691288143e-28
episode: 9193/10000, score: 9, eps: 3.4228214889541514e-28
episode: 9194/10000, score: 10, eps: 3.4194002073242013e-28
episode: 9195/10000, score: 10, eps: 3.415982345436715e-28
episode: 9196/10000, score: 9, eps: 3.4129091907925673e-28
episode: 9197/10000, score: 9, eps: 3.409838800881522e-28
episode: 9198/10000, score: 9, eps: 3.406771173216314e-28
episode: 9199/10000, score: 8, eps: 3.4040467099829143e-28
episode: 9200/10000, score: 8, eps: 3.401324425557404e-28
episode: 9201/10000, score: 10, eps: 3.397924631319751e-28
episode: 9202/10000, score: 10, eps: 3.3945282353468364e-28
episode: 9203/10000, score: 10, eps: 3.3911352342419233e-28
episode: 9204/10000, score: 10, eps: 3.3877456246116715e-28
episode: 9205/10000, score: 8, eps: 3.385036376491067e-28
episode: 9206/10000, score: 9, eps: 3.381991062081021e

episode: 9330/10000, score: 10, eps: 3.0103573338936494e-28
episode: 9331/10000, score: 8, eps: 3.007949890758029e-28
episode: 9332/10000, score: 10, eps: 3.0049432940838315e-28
episode: 9333/10000, score: 10, eps: 3.0019397026537e-28
episode: 9334/10000, score: 10, eps: 2.9989391134637434e-28
episode: 9335/10000, score: 10, eps: 2.995941523513071e-28
episode: 9336/10000, score: 10, eps: 2.9929469298037927e-28
episode: 9337/10000, score: 10, eps: 2.989955329341016e-28
episode: 9338/10000, score: 10, eps: 2.9869667191328422e-28
episode: 9339/10000, score: 9, eps: 2.9842795241427744e-28
episode: 9340/10000, score: 10, eps: 2.9812965871863667e-28
episode: 9341/10000, score: 10, eps: 2.9783166318249513e-28
episode: 9342/10000, score: 9, eps: 2.9756372188001557e-28
episode: 9343/10000, score: 8, eps: 2.9732575420369224e-28
episode: 9344/10000, score: 10, eps: 2.970285622104051e-28
episode: 9345/10000, score: 17, eps: 2.965240174115833e-28
episode: 9346/10000, score: 10, eps: 2.9622762679440

episode: 9475/10000, score: 9, eps: 2.602965876967786e-28
episode: 9476/10000, score: 8, eps: 2.60088423295091e-28
episode: 9477/10000, score: 10, eps: 2.598284518803813e-28
episode: 9478/10000, score: 8, eps: 2.5962066185629495e-28
episode: 9479/10000, score: 9, eps: 2.593870967022577e-28
episode: 9480/10000, score: 8, eps: 2.591796596387591e-28
episode: 9481/10000, score: 8, eps: 2.5897238846684056e-28
episode: 9482/10000, score: 8, eps: 2.587652830538352e-28
episode: 9483/10000, score: 8, eps: 2.5855834326718237e-28
episode: 9484/10000, score: 10, eps: 2.5829990124414806e-28
episode: 9485/10000, score: 10, eps: 2.5804171754686895e-28
episode: 9486/10000, score: 10, eps: 2.5778379191713547e-28
episode: 9487/10000, score: 10, eps: 2.5752612409699614e-28
episode: 9488/10000, score: 8, eps: 2.5732017529061367e-28
episode: 9489/10000, score: 8, eps: 2.5711439118562217e-28
episode: 9490/10000, score: 9, eps: 2.568830807731416e-28
episode: 9491/10000, score: 9, eps: 2.5665197845677998e-28


episode: 9761/10000, score: 9, eps: 1.996984446410491e-28
episode: 9762/10000, score: 10, eps: 1.9949883603674854e-28
episode: 9763/10000, score: 9, eps: 1.9931935888714108e-28
episode: 9764/10000, score: 10, eps: 1.9912012919805133e-28
episode: 9765/10000, score: 9, eps: 1.9894099274829604e-28
episode: 9766/10000, score: 10, eps: 1.987421412551258e-28
episode: 9767/10000, score: 8, eps: 1.9858320317879308e-28
episode: 9768/10000, score: 10, eps: 1.9838470931422993e-28
episode: 9769/10000, score: 9, eps: 1.982062344776807e-28
episode: 9770/10000, score: 10, eps: 1.9800811741222798e-28
episode: 9771/10000, score: 10, eps: 1.9781019837471181e-28
episode: 9772/10000, score: 10, eps: 1.9761247716719333e-28
episode: 9773/10000, score: 10, eps: 1.9741495359193156e-28
episode: 9774/10000, score: 9, eps: 1.9723735118650174e-28
episode: 9775/10000, score: 9, eps: 1.9705990855931486e-28
episode: 9776/10000, score: 10, eps: 1.9686293730407138e-28
episode: 9777/10000, score: 9, eps: 1.966858315146

episode: 9901/10000, score: 8, eps: 1.752479663343764e-28
episode: 9902/10000, score: 9, eps: 1.7509030623922472e-28
episode: 9903/10000, score: 9, eps: 1.7493278798141426e-28
episode: 9904/10000, score: 9, eps: 1.7477541143334255e-28
episode: 9905/10000, score: 8, eps: 1.746356400315249e-28
episode: 9906/10000, score: 10, eps: 1.7446108295657878e-28
episode: 9907/10000, score: 10, eps: 1.742867003601779e-28
episode: 9908/10000, score: 10, eps: 1.7411249206792212e-28
episode: 9909/10000, score: 9, eps: 1.739558534909349e-28
episode: 9910/10000, score: 9, eps: 1.7379935583229025e-28
episode: 9911/10000, score: 9, eps: 1.7364299896521234e-28
episode: 9912/10000, score: 10, eps: 1.7346943408476318e-28
episode: 9913/10000, score: 9, eps: 1.7331337402851397e-28
episode: 9914/10000, score: 8, eps: 1.7317477184733156e-28
episode: 9915/10000, score: 9, eps: 1.7301897688104237e-28
episode: 9916/10000, score: 10, eps: 1.7284603574194231e-28
episode: 9917/10000, score: 10, eps: 1.7267326746617862